# Wafer Defect Classification - Complete Solution

**⚠️ SOLUTION NOTEBOOK** - This contains complete implementations of all exercises from the tutorial.

## Purpose

This solution notebook provides:
- ✅ Complete code implementations for all exercises
- 💡 Detailed explanations of design decisions
- 📊 Expected outputs and interpretations
- ⚠️ Common pitfalls and debugging tips
- 🎯 Production-ready best practices

## How to Use This Notebook

1. **Attempt exercises first** - Try the tutorial exercises before looking at solutions
2. **Compare approaches** - Your solution may differ but still be correct
3. **Learn from differences** - Understand why alternative approaches work
4. **Adapt to your needs** - Solutions are templates, not rigid requirements

## Business Context

In semiconductor manufacturing, wafer defect detection is critical for:
- **Quality Control**: Early detection prevents defective dies from reaching customers
- **Cost Reduction**: Identifying process issues before they impact entire lots ($10K-$50K per lot)
- **Process Optimization**: Understanding defect patterns to improve manufacturing yield

**Industry Standards**:
- Target defect detection rate: >95%
- Acceptable false positive rate: <5%
- Inspection throughput: >100 wafers/hour
- Cost per false negative: $100-$500 (scrapped dies shipped)
- Cost per false positive: $10-$50 (good wafer unnecessarily scrapped)

## Setup and Imports

In [ ]:
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import json

# Import our wafer defect pipeline
from wafer_defect_pipeline import (
    WaferDefectPipeline, 
    generate_synthetic_wafer_defects,
    load_dataset
)

# Set random seed for reproducibility
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

# Configure plotting
plt.style.use('default')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 6)

print("✅ Environment setup complete")
print(f"NumPy version: {np.__version__}")
print(f"Pandas version: {pd.__version__}")

## Exercise 1: Data Generation and Exploration

**Task**: Generate synthetic wafer defect data and perform exploratory analysis.

**Requirements**:
1. Generate 1000 wafer samples with 20% defect rate
2. Calculate class distribution
3. Visualize feature distributions for defective vs. non-defective wafers
4. Identify top 3 most discriminative features

**Solution Approach**:
- Use the provided synthetic data generator
- Compare feature distributions using violin plots
- Calculate correlation with target variable
- Statistical tests for feature importance

In [ ]:
# Solution 1.1: Generate synthetic data
print("📊 Generating synthetic wafer defect data...\n")

# Generate 1000 samples with 20% defect rate
df = generate_synthetic_wafer_defects(
    n_samples=1000,
    defect_rate=0.20,
    seed=RANDOM_SEED
)

print(f"Dataset shape: {df.shape}")
print(f"\nColumns: {list(df.columns)}")
print(f"\nFirst few rows:")
display(df.head())

# Solution 1.2: Calculate class distribution
print("\n" + "="*60)
print("CLASS DISTRIBUTION ANALYSIS")
print("="*60)

class_counts = df['defect'].value_counts()
class_pct = df['defect'].value_counts(normalize=True) * 100

print(f"\nNon-defective wafers (0): {class_counts[0]:,} ({class_pct[0]:.1f}%)")
print(f"Defective wafers (1): {class_counts[1]:,} ({class_pct[1]:.1f}%)")
print(f"\nImbalance ratio: {class_counts[0] / class_counts[1]:.2f}:1")

# Manufacturing context
print("\n💡 Manufacturing Insight:")
print(f"   This {class_pct[1]:.1f}% defect rate is realistic for modern fabs")
print(f"   (Typical range: 5-30% depending on process maturity)")

In [ ]:
# Solution 1.3: Visualize feature distributions
print("\n📊 Visualizing feature distributions...")

# Get numeric features (exclude wafer_id and target)
feature_cols = [col for col in df.columns if col not in ['defect', 'wafer_id']]

# Create violin plots for each feature
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

for idx, feature in enumerate(feature_cols):
    ax = axes[idx]
    
    # Violin plot with defect grouping
    parts = ax.violinplot(
        [df[df['defect']==0][feature], df[df['defect']==1][feature]],
        positions=[0, 1],
        showmeans=True,
        showmedians=True
    )
    
    ax.set_xticks([0, 1])
    ax.set_xticklabels(['Non-Defective', 'Defective'])
    ax.set_ylabel(feature)
    ax.set_title(f'{feature} Distribution')
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('feature_distributions.png', dpi=100, bbox_inches='tight')
plt.show()

print("✅ Feature distribution plots saved as 'feature_distributions.png'")

In [ ]:
# Solution 1.4: Identify most discriminative features
print("\n" + "="*60)
print("FEATURE IMPORTANCE ANALYSIS")
print("="*60)

# Calculate correlation with target
correlations = df[feature_cols].corrwith(df['defect']).abs().sort_values(ascending=False)

print("\nFeature correlations with defect status:")
for feature, corr in correlations.items():
    print(f"  {feature:25s}: {corr:.4f}")

# Top 3 features
top_3 = correlations.head(3)
print(f"\n🎯 Top 3 Most Discriminative Features:")
for i, (feature, corr) in enumerate(top_3.items(), 1):
    print(f"  {i}. {feature}: {corr:.4f}")

# Manufacturing interpretation
print("\n💡 Manufacturing Interpretation:")
print("   - Higher correlations indicate stronger defect indicators")
print("   - These features should be prioritized in process control")
print("   - Real-world: align with physical defect mechanisms")

# Statistical significance testing
from scipy import stats

print("\n📊 Statistical Significance (t-tests):")
for feature in top_3.index:
    non_defect = df[df['defect']==0][feature]
    defect = df[df['defect']==1][feature]
    t_stat, p_value = stats.ttest_ind(non_defect, defect)
    print(f"  {feature:25s}: p-value = {p_value:.2e} {'***' if p_value < 0.001 else '**' if p_value < 0.01 else '*' if p_value < 0.05 else 'ns'}")

print("\n  *** p < 0.001 (highly significant)")
print("  **  p < 0.01  (very significant)")
print("  *   p < 0.05  (significant)")
print("  ns  p >= 0.05 (not significant)")

### Exercise 1 Key Takeaways

**✅ What You Learned**:
1. **Class Imbalance**: 20% defect rate creates 4:1 imbalance - requires special handling
2. **Feature Discrimination**: Some features are much better defect predictors than others
3. **Statistical Validation**: P-values confirm which differences are meaningful vs. random
4. **Manufacturing Context**: Feature importance should align with physical defect mechanisms

**⚠️ Common Pitfalls**:
- Ignoring class imbalance leads to biased models
- Using all features equally when some are noise
- Not validating statistical significance
- Forgetting to set random seed (results not reproducible)

**🎯 Production Considerations**:
- Real wafer data may have >100 features
- Feature selection reduces overfitting and computation
- Domain expert input crucial for feature interpretation
- Monitor feature distributions for data drift

## Exercise 2: Model Training and Comparison

**Task**: Train multiple classification models and compare their performance.

**Requirements**:
1. Train 5 different models: Logistic Regression, Linear SVM, Decision Tree, Random Forest, Gradient Boosting
2. Use 80/20 train/test split with stratification
3. Compare models using ROC AUC, PR AUC, and F1 score
4. Create ROC curve comparison plot
5. Recommend best model for production

**Solution Approach**:
- Use WaferDefectPipeline for consistent preprocessing
- Evaluate all models on same test set
- Consider multiple metrics (no single metric tells full story)
- Balance accuracy vs. interpretability vs. speed

In [ ]:
# Solution 2.1: Prepare train/test split
from sklearn.model_selection import train_test_split

print("📊 Preparing train/test split...\n")

# Separate features and target
y = df['defect'].to_numpy()
X = df.drop(columns=['defect', 'wafer_id'])

# Stratified split to maintain class distribution
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.20, 
    random_state=RANDOM_SEED,
    stratify=y  # Critical for imbalanced data!
)

print(f"Training set: {X_train.shape[0]:,} samples")
print(f"Test set:     {X_test.shape[0]:,} samples")
print(f"\nTrain defect rate: {y_train.mean()*100:.1f}%")
print(f"Test defect rate:  {y_test.mean()*100:.1f}%")
print("\n✅ Stratification maintained class balance")

In [ ]:
# Solution 2.2: Train multiple models
print("\n" + "="*60)
print("TRAINING MULTIPLE MODELS")
print("="*60 + "\n")

models_to_train = ['logistic', 'linear_svm', 'tree', 'rf', 'gb']
trained_models = {}
results = []

for model_name in models_to_train:
    print(f"Training {model_name}...", end=" ")
    
    # Initialize and train pipeline
    pipeline = WaferDefectPipeline(model=model_name)
    pipeline.fit(X_train, y_train)
    
    # Evaluate on test set
    metrics = pipeline.evaluate(X_test, y_test)
    
    # Store results
    trained_models[model_name] = pipeline
    results.append({
        'model': model_name,
        'roc_auc': metrics['roc_auc'],
        'pr_auc': metrics['pr_auc'],
        'f1': metrics['f1'],
        'precision': metrics['precision'],
        'recall': metrics['recall'],
        'pws': metrics['pws']
    })
    
    print(f"✅ ROC AUC: {metrics['roc_auc']:.4f}")

# Convert to DataFrame for easy comparison
results_df = pd.DataFrame(results)
print("\n" + "="*60)
print("MODEL COMPARISON RESULTS")
print("="*60)
display(results_df.round(4))

In [ ]:
# Solution 2.3: Visualize model comparison
from sklearn.metrics import roc_curve, auc

print("\n📊 Creating ROC curve comparison plot...")

plt.figure(figsize=(10, 8))

# Plot ROC curve for each model
for model_name, pipeline in trained_models.items():
    # Get predictions
    y_proba = pipeline.predict_proba(X_test)[:, 1]  # Positive class probabilities
    
    # Calculate ROC curve
    fpr, tpr, _ = roc_curve(y_test, y_proba)
    roc_auc = auc(fpr, tpr)
    
    # Plot
    plt.plot(fpr, tpr, linewidth=2, 
             label=f'{model_name} (AUC = {roc_auc:.3f})')

# Plot diagonal (random classifier)
plt.plot([0, 1], [0, 1], 'k--', linewidth=1, label='Random (AUC = 0.500)')

plt.xlabel('False Positive Rate', fontsize=12)
plt.ylabel('True Positive Rate (Recall)', fontsize=12)
plt.title('ROC Curve Comparison - All Models', fontsize=14, fontweight='bold')
plt.legend(loc='lower right', fontsize=10)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('roc_comparison.png', dpi=100, bbox_inches='tight')
plt.show()


In [ ]:
# Solution 2.4: Comprehensive model ranking
print("\n" + "="*60)
print("MODEL RANKING AND RECOMMENDATION")
print("="*60 + "\n")

# Rank by different metrics
print("Rankings by metric:")
print("\n1. ROC AUC (overall discrimination):")
top_roc = results_df.nlargest(3, 'roc_auc')[['model', 'roc_auc']]
for i, row in top_roc.iterrows():
    print(f"   {row['model']:15s}: {row['roc_auc']:.4f}")

print("\n2. PR AUC (precision-recall balance):")
top_pr = results_df.nlargest(3, 'pr_auc')[['model', 'pr_auc']]
for i, row in top_pr.iterrows():
    print(f"   {row['model']:15s}: {row['pr_auc']:.4f}")

print("\n3. F1 Score (precision-recall harmonic mean):")
top_f1 = results_df.nlargest(3, 'f1')[['model', 'f1']]
for i, row in top_f1.iterrows():
    print(f"   {row['model']:15s}: {row['f1']:.4f}")

print("\n4. PWS (manufacturing metric):")
top_pws = results_df.nlargest(3, 'pws')[['model', 'pws']]
for i, row in top_pws.iterrows():
    print(f"   {row['model']:15s}: {row['pws']:.1f}%")

# Overall recommendation
best_overall = results_df.loc[results_df['roc_auc'].idxmax()]

print("\n" + "="*60)
print("🎯 PRODUCTION RECOMMENDATION")
print("="*60)
print(f"\nBest Model: {best_overall['model'].upper()}")
print(f"\nPerformance Metrics:")
print(f"  - ROC AUC:   {best_overall['roc_auc']:.4f}")
print(f"  - PR AUC:    {best_overall['pr_auc']:.4f}")
print(f"  - F1 Score:  {best_overall['f1']:.4f}")
print(f"  - Precision: {best_overall['precision']:.4f}")
print(f"  - Recall:    {best_overall['recall']:.4f}")
print(f"  - PWS:       {best_overall['pws']:.1f}%")

print("\n💡 Rationale:")
if best_overall['model'] in ['rf', 'gb']:
    print("  ✅ Ensemble models typically provide best accuracy")
    print("  ✅ Handle non-linear relationships and interactions")
    print("  ✅ Built-in feature importance for interpretation")
    print("  ⚠️  Longer training time (acceptable for batch processing)")
    print("  ⚠️  Larger model size (manageable with modern hardware)")
elif best_overall['model'] == 'logistic':
    print("  ✅ Fast training and prediction")
    print("  ✅ Highly interpretable (feature coefficients)")
    print("  ✅ Small model size")
    print("  ⚠️  Assumes linear decision boundary")

print("\n🏭 Manufacturing Considerations:")
print("  - Prioritize recall if false negatives are costly (shipping defects)")
print("  - Prioritize precision if false positives are costly (scrapping good wafers)")
print("  - Use ROC AUC for balanced scenarios")
print("  - Monitor all metrics in production for drift detection")

### Exercise 2 Key Takeaways

**✅ What You Learned**:
1. **Model Selection**: Different algorithms have different strengths
2. **Multiple Metrics**: No single metric captures all aspects of performance
3. **Train/Test Split**: Stratification crucial for imbalanced data
4. **Ensemble Advantage**: Random Forest and Gradient Boosting often outperform simpler models

**⚠️ Common Pitfalls**:
- Using only accuracy (misleading for imbalanced data)
- Not stratifying splits (creates biased test sets)
- Overfitting to test set (use cross-validation for hyperparameter tuning)
- Ignoring computational constraints (training time, model size)

**🎯 Production Considerations**:
- Retraining frequency: Weekly/monthly based on data drift
- Model versioning: Track which model version is deployed
- A/B testing: Compare new models against baseline in production
- Fallback strategy: Keep previous version if new model fails

## Exercise 3: Manufacturing-Specific Metrics

**Task**: Calculate and interpret manufacturing-specific metrics for wafer defect classification.

**Requirements**:
1. Calculate PWS (Prediction Within Spec) for best model
2. Estimate financial loss from false positives and false negatives
3. Optimize decision threshold for different business scenarios
4. Create cost-benefit analysis visualization

**Solution Approach**:
- Assign realistic costs to false positives ($50) and false negatives ($200)
- Sweep decision threshold from 0.1 to 0.9
- Calculate total cost at each threshold
- Identify optimal threshold for different business priorities

In [ ]:
# Solution 3.1: Calculate PWS and financial metrics for best model
print("\n" + "="*60)
print("MANUFACTURING-SPECIFIC METRICS ANALYSIS")
print("="*60 + "\n")

# Use best model from Exercise 2
best_model = trained_models[best_overall['model']]

# Get predictions at default threshold (0.5)
y_pred = best_model.predict(X_test)
y_proba = best_model.predict_proba(X_test)[:, 1]  # Positive class probabilities

# Calculate confusion matrix elements
from sklearn.metrics import confusion_matrix
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()

print(f"Confusion Matrix (Threshold = 0.5):")
print(f"  True Negatives (TN):  {tn:4d} (correctly classified as non-defective)")
print(f"  False Positives (FP): {fp:4d} (good wafers incorrectly flagged as defective)")
print(f"  False Negatives (FN): {fn:4d} (defective wafers incorrectly passed as good) ⚠️")
print(f"  True Positives (TP):  {tp:4d} (correctly detected defective wafers)")

# PWS (Prediction Within Spec)
pws = (tp + tn) / len(y_test) * 100
print(f"\nPWS (Prediction Within Spec): {pws:.1f}%")

# Financial impact analysis
COST_FALSE_POSITIVE = 50    # Cost of unnecessarily scrapping a good wafer
COST_FALSE_NEGATIVE = 200   # Cost of shipping a defective wafer to customer

total_fp_cost = fp * COST_FALSE_POSITIVE
total_fn_cost = fn * COST_FALSE_NEGATIVE
total_cost = total_fp_cost + total_fn_cost

print(f"\n" + "="*60)
print("FINANCIAL IMPACT ANALYSIS")
print("="*60)
print(f"\nCost Parameters:")
print(f"  - Cost per False Positive: ${COST_FALSE_POSITIVE:,}")
print(f"  - Cost per False Negative: ${COST_FALSE_NEGATIVE:,}")
print(f"\nTotal Costs (test set = {len(y_test)} wafers):")
print(f"  - False Positive Cost: ${total_fp_cost:,} ({fp} wafers × ${COST_FALSE_POSITIVE})")
print(f"  - False Negative Cost: ${total_fn_cost:,} ({fn} wafers × ${COST_FALSE_NEGATIVE}) ⚠️")
print(f"  - TOTAL COST:          ${total_cost:,}")
print(f"\nCost per wafer inspected: ${total_cost/len(y_test):.2f}")

print(f"\n💡 Manufacturing Insight:")
print(f"   False negatives are {COST_FALSE_NEGATIVE/COST_FALSE_POSITIVE:.0f}x more costly than false positives")
print(f"   We should optimize threshold to minimize false negatives")

In [ ]:
# Solution 3.2: Threshold optimization for cost minimization
print("\n" + "="*60)
print("THRESHOLD OPTIMIZATION FOR COST MINIMIZATION")
print("="*60 + "\n")

# Sweep thresholds from 0.1 to 0.9
thresholds = np.linspace(0.1, 0.9, 50)
threshold_results = []

for threshold in thresholds:
    # Make predictions at this threshold
    y_pred_thresh = (y_proba >= threshold).astype(int)
    
    # Calculate confusion matrix
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred_thresh).ravel()
    
    # Calculate costs
    fp_cost = fp * COST_FALSE_POSITIVE
    fn_cost = fn * COST_FALSE_NEGATIVE
    total = fp_cost + fn_cost
    
    # Calculate metrics
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
    
    threshold_results.append({
        'threshold': threshold,
        'fp': fp,
        'fn': fn,
        'fp_cost': fp_cost,
        'fn_cost': fn_cost,
        'total_cost': total,
        'precision': precision,
        'recall': recall,
        'f1': f1
    })

threshold_df = pd.DataFrame(threshold_results)

# Find optimal threshold
optimal_idx = threshold_df['total_cost'].idxmin()
optimal_threshold = threshold_df.loc[optimal_idx, 'threshold']
optimal_cost = threshold_df.loc[optimal_idx, 'total_cost']

print(f"Optimal Threshold: {optimal_threshold:.3f}")
print(f"Optimal Total Cost: ${optimal_cost:,.0f}")
print(f"\nAt optimal threshold:")
print(f"  - False Positives: {threshold_df.loc[optimal_idx, 'fp']:.0f}")
print(f"  - False Negatives: {threshold_df.loc[optimal_idx, 'fn']:.0f}")
print(f"  - Precision: {threshold_df.loc[optimal_idx, 'precision']:.3f}")
print(f"  - Recall: {threshold_df.loc[optimal_idx, 'recall']:.3f}")
print(f"  - F1 Score: {threshold_df.loc[optimal_idx, 'f1']:.3f}")

# Compare to default threshold
default_idx = (threshold_df['threshold'] - 0.5).abs().idxmin()
default_cost = threshold_df.loc[default_idx, 'total_cost']
cost_savings = default_cost - optimal_cost
savings_pct = cost_savings / default_cost * 100

print(f"\nComparison to default threshold (0.5):")
print(f"  - Default cost: ${default_cost:,.0f}")
print(f"  - Optimal cost: ${optimal_cost:,.0f}")
print(f"  - Cost savings: ${cost_savings:,.0f} ({savings_pct:.1f}% reduction)")

print(f"\n💡 Manufacturing Insight:")
print(f"   Optimizing threshold reduces costs by {savings_pct:.1f}%")
print(f"   This translates to ${cost_savings/len(y_test):.2f} savings per wafer")

In [ ]:
# Solution 3.3: Visualize cost vs threshold
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Plot 1: Total cost vs threshold
ax1 = axes[0, 0]
ax1.plot(threshold_df['threshold'], threshold_df['total_cost'], 'b-', linewidth=2)
ax1.axvline(optimal_threshold, color='r', linestyle='--', linewidth=2, label=f'Optimal: {optimal_threshold:.3f}')
ax1.axvline(0.5, color='gray', linestyle=':', linewidth=1, label='Default: 0.500')
ax1.set_xlabel('Decision Threshold')
ax1.set_ylabel('Total Cost ($)')
ax1.set_title('Total Cost vs Threshold', fontweight='bold')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Plot 2: FP vs FN costs
ax2 = axes[0, 1]
ax2.plot(threshold_df['threshold'], threshold_df['fp_cost'], 'g-', linewidth=2, label='FP Cost')
ax2.plot(threshold_df['threshold'], threshold_df['fn_cost'], 'r-', linewidth=2, label='FN Cost')
ax2.axvline(optimal_threshold, color='k', linestyle='--', linewidth=1)
ax2.set_xlabel('Decision Threshold')
ax2.set_ylabel('Cost ($)')
ax2.set_title('FP vs FN Cost Components', fontweight='bold')
ax2.legend()
ax2.grid(True, alpha=0.3)

# Plot 3: Error counts
ax3 = axes[1, 0]
ax3.plot(threshold_df['threshold'], threshold_df['fp'], 'g-', linewidth=2, label='False Positives')
ax3.plot(threshold_df['threshold'], threshold_df['fn'], 'r-', linewidth=2, label='False Negatives')
ax3.axvline(optimal_threshold, color='k', linestyle='--', linewidth=1)
ax3.set_xlabel('Decision Threshold')
ax3.set_ylabel('Error Count')
ax3.set_title('Error Counts vs Threshold', fontweight='bold')
ax3.legend()
ax3.grid(True, alpha=0.3)

# Plot 4: Precision and Recall
ax4 = axes[1, 1]
ax4.plot(threshold_df['threshold'], threshold_df['precision'], 'b-', linewidth=2, label='Precision')
ax4.plot(threshold_df['threshold'], threshold_df['recall'], 'orange', linewidth=2, label='Recall')
ax4.plot(threshold_df['threshold'], threshold_df['f1'], 'purple', linewidth=2, label='F1 Score')
ax4.axvline(optimal_threshold, color='k', linestyle='--', linewidth=1)
ax4.set_xlabel('Decision Threshold')
ax4.set_ylabel('Metric Value')
ax4.set_title('Precision/Recall/F1 vs Threshold', fontweight='bold')
ax4.legend()
ax4.grid(True, alpha=0.3)
ax4.set_ylim([0, 1])

plt.tight_layout()
plt.savefig('threshold_optimization.png', dpi=100, bbox_inches='tight')
plt.show()

print("\n✅ Threshold optimization plots saved as 'threshold_optimization.png'")

### Exercise 3 Key Takeaways

**✅ What You Learned**:
1. **Cost Asymmetry**: False negatives often cost 2-10x more than false positives in manufacturing
2. **Threshold Optimization**: Default 0.5 threshold is rarely optimal for business metrics
3. **PWS Metric**: Manufacturing-specific metric that combines both error types
4. **Trade-offs**: Lower threshold → higher recall but lower precision (more false alarms)

**⚠️ Common Pitfalls**:
- Using same cost for FP and FN (unrealistic)
- Optimizing for accuracy instead of business cost
- Not considering downstream impact (warranty claims, customer satisfaction)
- Fixed threshold instead of adaptive threshold based on lot value

**🎯 Production Considerations**:
- Update cost estimates regularly (market conditions change)
- Different thresholds for different product lines (high-value vs commodity)
- Real-time threshold adjustment based on fab conditions
- Human-in-the-loop for borderline cases (probabilities near threshold)

## Exercise 4: Model Deployment and CLI Usage

**Task**: Save trained model and demonstrate production deployment using CLI.

**Requirements**:
1. Save best model with optimal threshold to disk
2. Load model and verify it produces same predictions
3. Demonstrate CLI usage for train/evaluate/predict commands
4. Create production deployment checklist

**Solution Approach**:
- Use joblib for model persistence
- Include threshold in saved metadata
- Test round-trip serialization
- Document CLI commands for production use

In [ ]:
# Solution 4.1: Save model with optimal threshold
print("\n" + "="*60)
print("MODEL PERSISTENCE AND DEPLOYMENT")
print("="*60 + "\n")

# Update best model with optimal threshold
best_model.fitted_threshold = optimal_threshold

# Save model
model_path = Path('wafer_defect_production_model.joblib')
best_model.save(model_path)

print(f"✅ Model saved to: {model_path.absolute()}")
print(f"\nModel metadata:")
print(f"  - Model type: {best_model.model_name}")
print(f"  - Decision threshold: {best_model.fitted_threshold:.3f}")
print(f"  - Training date: {best_model.metadata.trained_at}")
print(f"  - Number of features: {best_model.metadata.n_features_in}")
print(f"  - File size: {model_path.stat().st_size / 1024:.1f} KB")

In [ ]:
# Solution 4.2: Load model and verify predictions
print("\n" + "="*60)
print("MODEL LOADING AND VERIFICATION")
print("="*60 + "\n")

# Load model from disk
loaded_model = WaferDefectPipeline.load(model_path)

print(f"✅ Model loaded from: {model_path}")
print(f"\nLoaded model metadata:")
print(f"  - Model type: {loaded_model.model_name}")
print(f"  - Decision threshold: {loaded_model.fitted_threshold:.3f}")

# Verify predictions match
y_pred_original = best_model.predict(X_test[:10])
y_pred_loaded = loaded_model.predict(X_test[:10])

predictions_match = np.array_equal(y_pred_original, y_pred_loaded)

print(f"\nPrediction verification (first 10 samples):")
print(f"  - Original predictions: {y_pred_original}")
print(f"  - Loaded predictions:   {y_pred_loaded}")
print(f"  - Predictions match: {'✅ YES' if predictions_match else '❌ NO'}")

if predictions_match:
    print("\n✅ Model serialization verified successfully")
else:
    print("\n⚠️ WARNING: Model predictions differ after loading!")

In [ ]:
# Solution 4.3: Demonstrate CLI usage
print("\n" + "="*60)
print("CLI USAGE EXAMPLES")
print("="*60 + "\n")

cli_examples = [
    {
        'title': 'Training a Model',
        'command': 'python wafer_defect_pipeline.py train --dataset synthetic_wafer --model rf --min-precision 0.85 --save production_model.joblib',
        'description': 'Train Random Forest with minimum 85% precision constraint'
    },
    {
        'title': 'Evaluating a Model',
        'command': 'python wafer_defect_pipeline.py evaluate --model-path production_model.joblib --dataset synthetic_wafer',
        'description': 'Evaluate saved model on test data'
    },
    {
        'title': 'Making Predictions',
        'command': '''python wafer_defect_pipeline.py predict --model-path production_model.joblib --input-json '{"center_density":0.12, "edge_density":0.05, "defect_area_ratio":0.08, "defect_spread":2.5, "total_pixels":3000, "defect_pixels":240}' ''',
        'description': 'Predict defect status for single wafer'
    },
    {
        'title': 'High Recall Training',
        'command': 'python wafer_defect_pipeline.py train --dataset synthetic_wafer --model gb --min-recall 0.95 --save high_recall_model.joblib',
        'description': 'Train Gradient Boosting optimized for catching 95%+ of defects'
    },
    {
        'title': 'Imbalanced Data Handling',
        'command': 'python wafer_defect_pipeline.py train --dataset synthetic_wafer_1000_0.05 --model logistic --use-smote --save balanced_model.joblib',
        'description': 'Train with SMOTE oversampling for highly imbalanced data (5% defect rate)'
    }
]

for i, example in enumerate(cli_examples, 1):
    print(f"{i}. {example['title']}")
    print(f"   {example['description']}")
    print(f"\n   $ {example['command']}\n")

print("\n💡 All CLI commands return JSON output for programmatic consumption")

### Production Deployment Checklist

Use this checklist when deploying wafer defect classifier to production:

#### Pre-Deployment (Before Production)
- [ ] **Model Training**
  - Train on full production dataset (not synthetic)
  - Use cross-validation for hyperparameter tuning
  - Document training data date range and characteristics
  - Save training logs and metrics

- [ ] **Model Validation**
  - Test on held-out validation set
  - Verify performance meets business requirements
  - Test on edge cases and rare defect types
  - Get sign-off from domain experts

- [ ] **Threshold Optimization**
  - Calculate real cost estimates (FP and FN)
  - Optimize threshold for business objectives
  - Document threshold choice rationale
  - Get approval from manufacturing management

#### Deployment (Going Live)
- [ ] **Model Packaging**
  - Save model with optimal threshold
  - Include metadata (version, date, features, threshold)
  - Create model card with performance metrics
  - Version control model file (Git LFS or MLflow)

- [ ] **Integration Testing**
  - Test CLI interface with production data format
  - Verify predictions match validation results
  - Test error handling (missing features, invalid inputs)
  - Load testing for throughput requirements

- [ ] **Infrastructure**
  - Deploy model to production server
  - Set up monitoring and logging
  - Configure alerts for prediction errors
  - Create backup/rollback procedure

#### Post-Deployment (In Production)
- [ ] **Monitoring**
  - Track prediction distribution (class balance)
  - Monitor prediction latency and throughput
  - Alert on unusual patterns (data drift)
  - Log all predictions for auditing

- [ ] **Performance Tracking**
  - Compare predictions vs actual outcomes (ground truth)
  - Calculate production ROC AUC, precision, recall
  - Track false positive and false negative rates
  - Measure financial impact (cost savings)

- [ ] **Maintenance**
  - Schedule regular model retraining (weekly/monthly)
  - Review and update cost estimates quarterly
  - Re-optimize threshold if business conditions change
  - Document lessons learned and model improvements

#### Documentation
- [ ] **Technical Documentation**
  - Model architecture and hyperparameters
  - Feature engineering pipeline
  - Training procedure and data requirements
  - API/CLI usage guide

- [ ] **Business Documentation**
  - Model purpose and scope
  - Performance metrics and targets
  - Cost-benefit analysis
  - Limitations and failure modes

- [ ] **Operational Documentation**
  - Deployment procedure
  - Monitoring and alerting setup
  - Troubleshooting guide
  - Rollback procedure

## Summary and Next Steps

### What You Accomplished

In this solution notebook, you learned how to:

1. **Data Analysis** ✅
   - Generate realistic synthetic wafer defect data
   - Perform exploratory data analysis
   - Identify discriminative features
   - Handle class imbalance

2. **Model Development** ✅
   - Train and compare multiple classification algorithms
   - Evaluate using multiple metrics (ROC AUC, PR AUC, F1)
   - Select best model based on comprehensive analysis
   - Understand ensemble model advantages

3. **Manufacturing Metrics** ✅
   - Calculate PWS (Prediction Within Spec)
   - Estimate financial impact of errors
   - Optimize decision threshold for cost minimization
   - Balance precision and recall for business objectives

4. **Production Deployment** ✅
   - Save and load models with metadata
   - Verify model serialization
   - Use CLI interface for production workflows
   - Follow deployment best practices

### Key Takeaways

**Technical**:
- Ensemble models (RF, GB) typically outperform simpler models
- Threshold optimization can reduce costs by 10-30%
- Multiple metrics needed for comprehensive evaluation
- Stratified splitting crucial for imbalanced data

**Manufacturing**:
- False negatives usually cost 2-10x more than false positives
- PWS metric aligns with manufacturing quality standards
- Real-time monitoring essential for production deployment
- Domain expert validation critical for model acceptance

### Next Steps

To further improve your wafer defect classifier:

1. **Real Data Integration** 🔄
   - Replace synthetic data with actual wafer map images
   - Feature engineering from spatial patterns
   - Handle missing data and outliers

2. **Deep Learning** 🧠
   - Implement CNN for spatial pattern recognition
   - Transfer learning from pretrained models
   - Compare to classical ML baseline

3. **Advanced Techniques** 🚀
   - Hyperparameter optimization (Optuna, Grid Search)
   - Model ensemble (stacking, voting)
   - Online learning for continuous improvement

4. **Production MLOps** 🏭
   - MLflow experiment tracking
   - Model versioning and registry
   - A/B testing framework
   - Drift detection and alerting

### Related Modules

Continue your learning with these modules:
- **Module 6.2**: CNN for wafer map image classification
- **Module 9.1**: MLOps with MLflow
- **Module 10.2**: Testing and quality assurance
- **Module 5.2**: Time series for equipment drift monitoring

---

**🎉 Congratulations!** You've completed the wafer defect classification solution. You now have the skills to build production-ready classification systems for semiconductor manufacturing.